In [2]:
import os
import pandas as pd;pd.__version__

'0.24.2'

In [57]:
trade_folder = r'C:\Users\bhou\proj\trade\DataOutput'
data_folder = r'C:\Users\bhou\proj\trade\RawData'
ticker = 'GBPUSD'

In [58]:
def get_filter_time_frame(trade_time_frame):
    if trade_time_frame == 'Id3':
        return 'Id2'
    if trade_time_frame == 'Id2':
        return 'Id1'
    if trade_time_frame == 'Id1':
        return 'Daily'
    raise KeyError(trade_time_frame + 'is not supported.')

In [66]:
df = pd.read_csv(os.path.join(data_folder, ticker +'.csv'), parse_dates=['TickDateTime']).set_index('TickDateTime')
df = df.loc[~df.index.duplicated(keep='first')]

In [67]:
trade_df = pd.read_csv(os.path.join(trade_folder, ticker + '.csv'), names=['Direction', 'TimeFrame', 'Ticker', 'Quantity', 'EntryTime', 'EntryPrice', 'PeelTime', 'PeelPrice', 'StopPrice','TargetPrice','ATR','FtATR','BgMarketEnv','FtMarketEnv','BgVPOC','BgVAH','BgVAL','FtVPOC','FtVAH','FtVAL','NextBar1Price','NextBar2Price','NextBar3Price','MAE', 'MFE'], parse_dates=['EntryTime', 'PeelTime']);trade_df.head()

,Direction,TimeFrame,Ticker,Quantity,EntryTime,EntryPrice,PeelTime,PeelPrice,StopPrice,TargetPrice,...,BgVAH,BgVAL,FtVPOC,FtVAH,FtVAL,NextBar1Price,NextBar2Price,NextBar3Price,MAE,MFE
0,Short,ID2,GBPUSD,1,2019-10-09 04:29:49,1.22720,2019-10-09 05:30:34,1.22244,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Long,ID2,GBPUSD,1,2019-10-09 06:01:46,1.22285,2019-10-09 06:20:41,1.22150,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Long,ID2,GBPUSD,1,2019-10-13 21:20:10,1.25927,2019-10-14 02:19:57,1.25801,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Short,ID2,GBPUSD,1,2019-10-14 10:29:57,1.26225,2019-10-14 11:07:32,1.25769,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Long,ID2,GBPUSD,1,2019-10-15 11:51:01,1.27239,2019-10-15 12:54:56,1.27525,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
for row_index, row in trade_df[trade_df['MFE'].isnull()].iterrows():
    trade_time_frame = row['TimeFrame'].lower().title()
    filter_time_frame = get_filter_time_frame(trade_time_frame)
    trade_entry_time = row['EntryTime']
    trade_entry_price = row['EntryPrice']
    trade_exit_time = row['PeelTime']
    
    try:
        df.loc[trade_entry_time]
    except KeyError:
        print('Unable to find entry time, use the next available')
        trade_entry_time = df.loc[trade_entry_time:].index[0]
    
    try:
        df.loc[trade_exit_time]
    except KeyError:
        print('Unable to find exit time, use the next available')
        trade_exit_time = df.loc[trade_exit_time:].index[0]
    
    # ATR of the trade
    _atr = df.loc[trade_entry_time, trade_time_frame + 'Atr']
    trade_df.loc[row_index, 'ATR'] = _atr
    trade_df.loc[row_index, 'FtATR'] = df.loc[trade_entry_time, filter_time_frame + 'Atr']
    
    # Background / Filter MarketEnv
    trade_df.loc[row_index, 'BgMarketEnv'] = df.loc[trade_entry_time, trade_time_frame + 'MarketEnv']
    trade_df.loc[row_index, 'FtMarketEnv'] = df.loc[trade_entry_time, filter_time_frame + 'MarketEnv']
    
    # Background / Filter VPOC
    trade_df.loc[row_index, 'BgVPOC'] = df.loc[trade_entry_time, trade_time_frame + 'VPOC']
    trade_df.loc[row_index, 'BgVAL'] = df.loc[trade_entry_time, trade_time_frame + 'VAL']
    trade_df.loc[row_index, 'BgVAH'] = df.loc[trade_entry_time, trade_time_frame + 'VAH']
    trade_df.loc[row_index, 'FtVPOC'] = df.loc[trade_entry_time, filter_time_frame + 'VPOC']
    trade_df.loc[row_index, 'FtVAL'] = df.loc[trade_entry_time, filter_time_frame + 'VAL']
    trade_df.loc[row_index, 'FtVAH'] = df.loc[trade_entry_time, filter_time_frame + 'VAH']
    
    # Stop Price of the trade based on ATR
    atr_ratio = 0.4
    stop_amount = _atr * atr_ratio
    if row['Direction'] == 'Long':
        stop_amount = -1 * stop_amount
    trade_df.loc[row_index, 'StopPrice'] = trade_entry_price + stop_amount
    
    # Target Price is based on Envelope price, if it is not as 1 to 1, then move to filter Envelope
    bg_envelope_top = df.loc[trade_entry_time, trade_time_frame + 'EnvelopeHigh']
    bg_envelope_bottom = df.loc[trade_entry_time, trade_time_frame + 'EnvelopeLow']
    ft_envelope_top = df.loc[trade_entry_time, filter_time_frame + 'EnvelopeHigh']
    ft_envelope_bottom = df.loc[trade_entry_time, filter_time_frame + 'EnvelopeLow']
    if row['Direction'] == 'Long':
        target_price = bg_envelope_top if (bg_envelope_top - trade_entry_price) >= stop_amount else ft_envelope_top
    else:
        target_price = bg_envelope_bottom if (trade_entry_price - bg_envelope_bottom) >= stop_amount else ft_envelope_bottom
    trade_df.loc[row_index, 'TargetPrice'] = target_price
    
    bg_bar_number_name = trade_time_frame + 'BarNumber'
    bg_bar_number = df.loc[trade_exit_time, bg_bar_number_name]
    
    # Next Close Bar 1
    trade_df.loc[row_index, 'NextBar1Price'] = df[df[bg_bar_number_name] == bg_bar_number + 1].iloc[-1].loc['ClosePrice']
    
    # Next Close Bar 2
    trade_df.loc[row_index, 'NextBar2Price'] = df[df[bg_bar_number_name] == bg_bar_number + 2].iloc[-1].loc['ClosePrice']
    
    # Next Close Bar 3
    trade_df.loc[row_index, 'NextBar3Price'] = df[df[bg_bar_number_name] == bg_bar_number + 3].iloc[-1].loc['ClosePrice']
    
    
    bg_entry_bar_number = df.loc[trade_entry_time, bg_bar_number_name]
    _df =  df[df[bg_bar_number_name].isin(list(range(bg_entry_bar_number, bg_bar_number + 4)))]
    min_value = _df['LowPrice'].min()
    max_value = _df['HighPrice'].max()
    if row['Direction'] == 'Long':
        trade_df.loc[row_index, 'MAE'] = min_value
        trade_df.loc[row_index, 'MFE'] = max_value
    else:
        trade_df.loc[row_index, 'MAE'] = max_value
        trade_df.loc[row_index, 'MFE'] = min_value

In [69]:
trade_df.to_csv(ticker + '_processed.csv', index=False);trade_df.head()

,Direction,TimeFrame,Ticker,Quantity,EntryTime,EntryPrice,PeelTime,PeelPrice,StopPrice,TargetPrice,...,BgVAH,BgVAL,FtVPOC,FtVAH,FtVAL,NextBar1Price,NextBar2Price,NextBar3Price,MAE,MFE
0,Short,ID2,GBPUSD,1,2019-10-09 04:29:49,1.22720,2019-10-09 05:30:34,1.22244,1.22836,1.2212,...,1.2273,1.2221,1.2205,1.2217,1.2203,1.22241,1.22194,1.22261,1.22759,1.22054
1,Long,ID2,GBPUSD,1,2019-10-09 06:01:46,1.22285,2019-10-09 06:20:41,1.22150,1.22169,1.2272,...,1.2242,1.2220,1.2224,1.2242,1.2198,1.22194,1.22261,1.22203,1.22054,1.22353
2,Long,ID2,GBPUSD,1,2019-10-13 21:20:10,1.25927,2019-10-14 02:19:57,1.25801,1.25751,1.2656,...,1.2641,1.2614,1.2668,1.2674,1.2650,1.25904,1.25683,1.25536,1.25516,1.26163
3,Short,ID2,GBPUSD,1,2019-10-14 10:29:57,1.26225,2019-10-14 11:07:32,1.25769,1.26365,1.2565,...,1.2628,1.2597,1.2555,1.2577,1.2550,1.25627,1.25530,1.26047,1.26348,1.25332
4,Long,ID2,GBPUSD,1,2019-10-15 11:51:01,1.27239,2019-10-15 12:54:56,1.27525,1.27055,1.2770,...,1.2752,1.2729,1.2737,1.2789,1.2724,1.27730,1.27765,1.27812,1.27135,1.27922


In [70]:
import glob, os
os.chdir(".")
process_list = []
for file in glob.glob("*_processed.csv"):
    process_list.append(pd.read_csv(file))

In [71]:
df = pd.concat(process_list, axis=0);df.shape

(240, 25)

In [72]:
df.drop_duplicates().to_csv('final_trades.csv', index=False)